In [ ]:
from ollama import chat
from pydantic import BaseModel,Field
from pymupdf4llm import to_markdown
from typing import Annotated
from annotated_types import Len

class ResumeData(BaseModel):
    name: str
    email: str
    skills: Annotated[list[str], Len(min_length=1, max_length=5)] = Field(
        description="List of exactly 5 top technical skills."
    )
    experience: int
    summary: str
    strengths: list[str]
    fit_explanation: str

In [119]:
pdf_path = "data/Alex_Morgan_Senior_Software_Engineer.pdf"
extracted_text = to_markdown(pdf_path)

In [120]:
prompt = f"""
    You are an expert resume parser. 
    Extract the following information from the resume text provided.
    1. The candidate's full name
    2. The candidate's email address
    3. The candidate's Skills (as a list)
    4. The candidate's Years of Experience (as an integer)
    5. Summarize the candidate's professional background in one sentence.
    6. List the candidate's top strengths (as a list).
    7. Explain in two sentences why this candidate is a good fit for a senior software engineer role.
    Resume Text: {extracted_text}
    """

In [121]:
response = chat(model="llama3.2", messages=[
    {
        'role': "user",
        'content': prompt,
    }
],format=ResumeData.model_json_schema())


In [122]:
print(response['message']['content'])

{ "name": "Alex Morgan", "email": "alex.morgan@example.com", "skills": ["C#", ".NET 8", "React", "TypeScript", "SQL", "Azure", "Docker"], "experience": 10, "summary": "Senior Software Engineer with extensive experience in building scalable web applications and APIs.", "strengths": ["Leadership skills", "Technical expertise", "Problem-solving abilities"], "fit_explanation": "This candidate is a good fit for a senior software engineer role due to their decade-long experience in modernizing legacy systems and leading teams. Their technical expertise and problem-solving abilities make them well-suited to design and develop complex web applications and APIs."}


In [123]:
validated_model = ResumeData.model_validate_json(response['message']['content'])
pretty_json = validated_model.model_dump_json(indent=4)
print(pretty_json)

{
    "name": "Alex Morgan",
    "email": "alex.morgan@example.com",
    "skills": [
        "C#",
        ".NET 8",
        "React",
        "TypeScript",
        "SQL",
        "Azure",
        "Docker"
    ],
    "experience": 10,
    "summary": "Senior Software Engineer with extensive experience in building scalable web applications and APIs.",
    "strengths": [
        "Leadership skills",
        "Technical expertise",
        "Problem-solving abilities"
    ],
    "fit_explanation": "This candidate is a good fit for a senior software engineer role due to their decade-long experience in modernizing legacy systems and leading teams. Their technical expertise and problem-solving abilities make them well-suited to design and develop complex web applications and APIs."
}
